In [1]:
from network import Network as network
from agent import Agent as agent

#BeamRider = network.make_bsc_img_mdl((2,84,84), 9, 0.00025)
#BeamRider = network.make_bsc_img_mdl((2,84,84), 4, 0.00025)
BeamRider = network.make_bsc_img_mdl((2,84,84), 6, 0.00025)

#BeamRider.load_weights("model-BeamRider-v0-3475.h5")
#BeamRider.load_weights("model-Breakout-v0-19475.h5")
BeamRider.load_weights("model-SpaceInvaders-v0-7975.h5")

#BeamRider_weights = BeamRider.get_weights()
#Breakout_weights = Breakout.get_weights()
#SpaceInvaders_weights = SpaceInvaders.get_weights()

#BeamRider2 = network.make_bsc_img_mdl((4,84,84), 9, 0.00025)
#BeamRider2 = network.make_bsc_img_mdl((4,84,84), 4, 0.00025)
BeamRider2 = network.make_bsc_img_mdl((4,84,84), 6, 0.00025)

#BeamRider_weights2 = BeamRider2.get_weights()
#Breakout_weights2 = Breakout2.get_weights()
#SpaceInvaders_weights2 = SpaceInvaders2.get_weights()



#print(type(BeamRider_weights), len(BeamRider_weights),len(BeamRider_weights[0]))
#print("First")
#print(BeamRider.get_layer("conv2d_2").get_weights())

weights1 = BeamRider.get_layer("conv2d_2").get_weights()
weights2 = BeamRider.get_layer("conv2d_3").get_weights()
weights3 = BeamRider.get_layer("flatten_1").get_weights()
weights4 = BeamRider.get_layer("dense_1").get_weights()
weights5 = BeamRider.get_layer("dense_2").get_weights()


BeamRider2.get_layer("conv2d_5").set_weights(weights1)
BeamRider2.get_layer("conv2d_6").set_weights(weights2)
BeamRider2.get_layer("flatten_2").set_weights(weights3)
BeamRider2.get_layer("dense_3").set_weights(weights4)
BeamRider2.get_layer("dense_4").set_weights(weights5)

BeamRider2.save_weights("./SpaceInvaders_ready.h5")

#print("Second")
#print(BeamRider2.get_layer("conv2d_5").get_weights())

#print(type(BeamRider_weights2), len(BeamRider_weights2),len(BeamRider_weights2[0]))
"""
BeamRider_weights2[1] = BeamRider_weights
BeamRider_weights2[2] = BeamRider_weights
BeamRider_weights2[3] = BeamRider_weights
BeamRider_weights2[4] = BeamRider_weights
BeamRider_weights2[5] = BeamRider_weights
BeamRider_weights2[6] = BeamRider_weights
BeamRider_weights2[7] = BeamRider_weights
BeamRider_weights2[8] = BeamRider_weights
BeamRider_weights2[9] = BeamRider_weights
"""
#BeamRider2.load_weights(BeamRider_weights2)

#print(BeamRider_weights)

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2, 84, 84)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 20, 20)        4128      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 9, 64)         20544     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 7, 64)         36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 5824)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               2982400   
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 3078      
Total para

'\nBeamRider_weights2[1] = BeamRider_weights\nBeamRider_weights2[2] = BeamRider_weights\nBeamRider_weights2[3] = BeamRider_weights\nBeamRider_weights2[4] = BeamRider_weights\nBeamRider_weights2[5] = BeamRider_weights\nBeamRider_weights2[6] = BeamRider_weights\nBeamRider_weights2[7] = BeamRider_weights\nBeamRider_weights2[8] = BeamRider_weights\nBeamRider_weights2[9] = BeamRider_weights\n'

In [ ]:
from network import Network as network
from agent import Agent as agent

BeamRider2 = network.make_bsc_img_mdl((4,84,84), 9, 0.00025)
BeamRider2 = network.make_bsc_img_mdl((4,84,84), 4, 0.00025)
BeamRider2 = network.make_bsc_img_mdl((4,84,84), 6, 0.00025)

BeamRider.load_weights("model-BeamRider-v0-3475.h5")
BeamRider.load_weights("model-Breakout-v0-19475.h5")
BeamRider.load_weights("model-SpaceInvaders-v0-7975.h5")